In [1]:
import os
import glob
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

# BGE-M3 모델 로드 (BAAI/bge-m3 사용)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 지정된 폴더와 내부 폴더 경로 설정
root_dir = './COMPANY_DATA2'  # 지정된 폴더 경로 (직급별 폴더들)

# Chroma 벡터 DB 초기화
db_dir = './company_chroma_db'
vectorstore = Chroma(persist_directory=db_dir, embedding_function=embedding_model)

# 청킹을 위한 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# 폴더 순회 및 파일 처리
def process_files_in_folder(folder_path: str, role: str):
    # 지정된 폴더 내의 모든 텍스트 파일 찾기
    txt_files = glob.glob(os.path.join(folder_path, "*.txt"))

    for txt_file in txt_files:
        print(f"Processing file: {txt_file}")
        with open(txt_file, 'r', encoding='utf-8') as f:
            text = f.read()

        # 문서 청킹
        chunks = text_splitter.split_text(text)

        # 청크를 Document 객체로 변환
        documents = []
        for chunk in chunks:
            doc = Document(
                page_content=chunk,
                metadata={
                    "sourcefile": os.path.basename(txt_file),  # 파일명
                    "role": role,  # 직급 권한 (폴더 이름)
                    "last_edit": "2025-08-19"  # 마지막 수정일 고정
                }
            )
            documents.append(doc)

        # 배치로 문서들 추가 (올바른 메서드 사용)
        if documents:
            vectorstore.add_documents(documents)
            print(f"Added {len(documents)} chunks from {os.path.basename(txt_file)}")

# 루트 폴더 내의 모든 직급 폴더 처리
def process_root_folder(root_dir: str):
    for role_folder in os.listdir(root_dir):
        role_folder_path = os.path.join(root_dir, role_folder)
        if os.path.isdir(role_folder_path):  # 내부 폴더만 처리
            print(f"Processing role folder: {role_folder}")
            process_files_in_folder(role_folder_path, role_folder)

# 데이터 처리 시작
if __name__ == "__main__":
    try:
        process_root_folder(root_dir)
        print("모든 문서가 성공적으로 벡터 데이터베이스에 저장되었습니다.")
    except Exception as e:
        print(f"오류 발생: {e}")

Processing role folder: backend
Processing file: ./COMPANY_DATA2\backend\01_backend__서비스_아키텍처_문서.txt
Added 4 chunks from 01_backend__서비스_아키텍처_문서.txt
Processing file: ./COMPANY_DATA2\backend\02_backend__보안_인증_가이드.txt
Added 4 chunks from 02_backend__보안_인증_가이드.txt
Processing file: ./COMPANY_DATA2\backend\03_backend__에러_핸들링_매뉴얼.txt
Added 4 chunks from 03_backend__에러_핸들링_매뉴얼.txt
Processing file: ./COMPANY_DATA2\backend\04_backend__배포_운영_가이드.txt
Added 4 chunks from 04_backend__배포_운영_가이드.txt
Processing file: ./COMPANY_DATA2\backend\05_backend__데이터베이스_스키마_&_변경_이력_문서.txt
Added 5 chunks from 05_backend__데이터베이스_스키마_&_변경_이력_문서.txt
Processing file: ./COMPANY_DATA2\backend\06_backend__API_정책_문서.txt
Added 6 chunks from 06_backend__API_정책_문서.txt
Processing file: ./COMPANY_DATA2\backend\07_backend__캐시_및_세션_관리_정책_문서.txt
Added 4 chunks from 07_backend__캐시_및_세션_관리_정책_문서.txt
Processing file: ./COMPANY_DATA2\backend\08_backend__장애_대응_매뉴얼_(Runbook).txt
Added 3 chunks from 08_backend__장애_대응_매뉴얼_(Runbook).txt


In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# BGE-M3 모델 로드 (BAAI/bge-m3 사용)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Chroma 벡터 DB 로드
db_dir = './company_chroma_db'
vectorstore = Chroma(persist_directory=db_dir, embedding_function=embedding_model)

# 검색할 쿼리
query = "코드노바의 캐시 만료 시간은 어떤 기준으로 설정해야 하나요?"

# 벡터 DB에서 유사한 문서 검색
results = vectorstore.similarity_search(query, k=3)  # k는 검색할 상위 결과의 수 (예: 3)

# 검색된 결과 출력
for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(f"Content: {result.page_content}")
    print(f"Metadata: {result.metadata}")
    print("=" * 50)


C:\Users\playdata\AppData\Local\Temp\ipykernel_9484\4218577003.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
C:\Users\playdata\AppData\Local\Temp\ipykernel_9484\4218577003.py:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=db_dir, embedding_function=embed

Result 1:
Content: <!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->
# 캐시 및 세션 관리 정책 문서
분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

## 1. 목적
본 문서는 코드노바의 백엔드 팀에서 캐시 및 세션 관리를 효과적으로 수행하기 위한 정책을 정의합니다. 이를 통해 시스템 성능을 최적화하고 사용자 경험을 향상시키는 것을 목표로 합니다.

## 2. 캐시 관리 정책

### 2.1 캐시 사용 원칙
- **데이터 일관성**: 캐시된 데이터는 항상 최신 상태를 유지해야 하며, 데이터 변경 시 적절한 캐시 무효화가 필요합니다.
- **캐시 만료**: 각 캐시 항목은 유효 기간을 설정하여 자동으로 만료되도록 합니다. 만료 시간은 데이터의 특성에 따라 조정합니다.
- **캐시 적중률 모니터링**: 캐시 적중률을 정기적으로 모니터링하여 성능 개선의 기회를 파악합니다.
Metadata: {'role': 'backend', 'last_edit': '2025-08-19', 'sourcefile': '07_backend__캐시_및_세션_관리_정책_문서.txt'}
Result 2:
Content: <!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->
# 보관·백업 정책 문서
분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

## 1. 목적
본 문서는 코드노바의 백엔드팀이 데이터 보관 및 백업을 효과적으로 수행하기 위한 정책을 제정하여 데이터의 안전성과 지속성을 보장하는 것을 목적으로 한다.

## 2. 데이터 분류
- **중요 데이터**: 비즈니스 운영에 필수적이며, 손실 시 심각한 영향을 미치는 데이터.
- **일반 데이터**: 비즈니스 운영에 필요하지만, 손실 시 즉각적인 영향을 미치지 않는 데이터.
- **임시 데이터**: 단기적으로 사용되며, 장기 보관이 필요 없는 데이터.

## 